Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# Accessing a Bundle Workflow in Python

In this guide, we'll explore how to access a bundle in Python and use it in your own application. We'll cover the following topics:

1. **Downloading the Bundle**: First, you'll need to download the bundle from its source. This can be done using the `download` API.

2. **Creating a `BundleWorkflow`**: Once you have the bundle, you can create a `BundleWorkflow` object by passing the path to the bundle file as an argument to `create_worflow`.

3. **Getting Properties from the Bundle**: You can then retrieve the properties of the bundle by directly accessing them. For example, to get the version of the bundle, you can use `workflow.version`.

4. **Using Pretrained Weights from the Bundle**: You can conveniently employ pretrained weights from the bundle and customize them using the `load` API.

5. **Updating Properties**: If you need to update any of the properties, you can do so by directly overwriting them. For example, to update the max epochs of the bundle, you can use `workflow.max_epochs = 10`.

6. **Using Components in Your Own Pipeline**: Finally, you can use the components from the bundle in your own pipeline by accessing them through the `BundleWorkflow` object.


## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[gdown, nibabel, tqdm, ignite]"

## Setup imports

In [8]:
import os
import tempfile
from pathlib import Path
from monai.networks.nets import SegResNet
from monai.transforms import MeanEnsembled, Compose
from monai.config import print_config
from monai.bundle import download, create_workflow, ConfigParser, load

print_config()

MONAI version: 1.2.0+107.g1ed4f94b
Numpy version: 1.22.2
Pytorch version: 1.13.1+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 1ed4f94ba3ab84cd5b7c14c840f1cf8b9269e266
MONAI __file__: /workspace/Code/MONAI/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.3.0
Nibabel version: 5.1.0
scikit-image version: 0.21.0
scipy version: 1.10.1
Pillow version: 9.2.0
Tensorboard version: 2.9.0
gdown version: 4.7.1
TorchVision version: 0.14.1+cu117
tqdm version: 4.65.0
lmdb version: 1.4.1
psutil version: 5.9.4
pandas version: 1.5.2
einops version: 0.6.1
transformers version: 4.21.3
mlflow version: 2.4.0
pynrrd version: 1.0.0
clearml version: 1.11.1rc1

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup data directory

You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.  
This allows you to save results and reuse downloads.  
If not specified a temporary directory will be used.

In [2]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/workspace/Data


## Downloading the Bundle

In [4]:
download(name="spleen_ct_segmentation", bundle_dir=root_dir)

name spleen_ct_segmentation
version None
bundle_dir /workspace/Data
source monaihosting
repo None
url None
remove_prefix monai_
progress True
2023-09-06 03:32:53,482 - INFO - --- input summary of monai.bundle.scripts.download ---
2023-09-06 03:32:53,483 - INFO - > name: 'spleen_ct_segmentation'
2023-09-06 03:32:53,483 - INFO - > bundle_dir: '/workspace/Data'
2023-09-06 03:32:53,483 - INFO - > source: 'monaihosting'
2023-09-06 03:32:53,484 - INFO - > remove_prefix: 'monai_'
2023-09-06 03:32:53,484 - INFO - > progress: True
2023-09-06 03:32:53,484 - INFO - ---


2023-09-06 03:32:54,228 - INFO - Expected md5 is None, skip md5 check for file /workspace/Data/spleen_ct_segmentation_v0.5.3.zip.
2023-09-06 03:32:54,228 - INFO - File exists: /workspace/Data/spleen_ct_segmentation_v0.5.3.zip, skipped downloading.
2023-09-06 03:32:54,230 - INFO - Writing into directory: /workspace/Data.


## Creating a `BundleWorkflow`
In this section, we demonstrate how to create and initialize a `BundleWorkflow` using the `create_workflow` function. This function supports the creation of both config-based bundles by providing the necessary config files and python-based bundles by specifying a bundle workflow name. The specified name should be a subclass of `BundleWorkflow` and be accessible for import.


In [5]:
config_file = Path(root_dir) / "spleen_ct_segmentation" / "configs" / "train.json"

train_workflow = create_workflow(config_file=config_file, workflow_type="train")

workflow_name None
config_file /workspace/Data/spleen_ct_segmentation/configs/train.json
workflow_type train
2023-09-06 03:33:02,472 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-09-06 03:33:02,473 - INFO - > config_file: PosixPath('/workspace/Data/spleen_ct_segmentation/configs/train.json')
2023-09-06 03:33:02,473 - INFO - > workflow_type: 'train'
2023-09-06 03:33:02,474 - INFO - ---


2023-09-06 03:33:02,474 - INFO - Setting logging properties based on config: /workspace/Data/spleen_ct_segmentation/configs/logging.conf.


## Getting Properties from the Bundle
To access properties from the bundle, please refer to the list of supported properties available [here](https://docs.monai.io/en/latest/mb_properties.html).

You can also utilize the `add_property` method of the `BundleWorkflow` object to add properties for application requirements checking and access.


In [14]:
# for existing properties
# Note that the properties got from `train_workflow` is already instantiated.
train_preprocessing = train_workflow.train_preprocessing

# for meta information
version = train_workflow.version

# add properties
train_workflow.add_property(name="lr_scheduler", required=True, config_id="lr_scheduler")
print(train_workflow.lr_scheduler)

monai.transforms.io.dictionary LoadImaged.__init__:image_only: Current default value of argument `image_only=False` has been deprecated since version 1.1. It will be changed to `image_only=True` in version 1.3.


## Utilizing Pretrained Weights from the Bundle

This function primarily serves to provide an instantiated network by loading pretrained weights from the bundle. You have the flexibility to directly update the parameters or filter the weights. Additionally, it's possible to use your own model instead of the one included in the bundle.


In [9]:
# directly get an instantiated network that loaded the weights.
model = load(name="brats_mri_segmentation", bundle_dir=root_dir, source="monaihosting")

# directly update the parameters for the model from the bundle.
model = load(name="brats_mri_segmentation", bundle_dir=root_dir, source="monaihosting", in_channels=3, out_channels=1)

# using `exclude_vars` to filter loading weights.
model = load(name="brats_mri_segmentation", bundle_dir=root_dir, source="monaihosting", copy_model_args={"exclude_vars": "convInit|conv_final"})

# pass model and return an instantiated network that loaded the weights.
my_model = SegResNet(blocks_down=[1, 2, 2, 4], blocks_up=[1, 1, 1], init_filters=16, in_channels=1, out_channels=3)
model = load(name="brats_mri_segmentation", bundle_dir=root_dir, source="monaihosting", model=my_model)

2023-09-06 04:06:53,867 - INFO - 'dst' model updated: 82 of 83 variables.


## Updating Properties
There are two primary methods for updating properties:

1. You can override them during the workflow creation process.
2. Alternatively, you can directly overwrite them after the workflow has been created.

In [6]:
# 1 override them when you create the workflow
dataset_dir = Path(root_dir) / "Task09_Spleen"
bundle_root = root_dir
override = {"epochs": 1, "dataset_dir": dataset_dir, "bundle_root": bundle_root}
train_workflow = create_workflow(config_file=config_file, workflow_type="train", **override)
print("max epochs:", train_workflow.max_epochs)

workflow_name None
config_file /workspace/Data/spleen_ct_segmentation/configs/train.json
workflow_type train
epochs 1
dataset_dir /workspace/Data/Task09_Spleen
bundle_root /workspace/Data
2023-09-05 10:32:00,679 - INFO - --- input summary of monai.bundle.scripts.run ---
2023-09-05 10:32:00,682 - INFO - > config_file: PosixPath('/workspace/Data/spleen_ct_segmentation/configs/train.json')
2023-09-05 10:32:00,682 - INFO - > workflow_type: 'train'
2023-09-05 10:32:00,684 - INFO - > epochs: 1
2023-09-05 10:32:00,684 - INFO - > dataset_dir: PosixPath('/workspace/Data/Task09_Spleen')
2023-09-05 10:32:00,685 - INFO - > bundle_root: '/workspace/Data'
2023-09-05 10:32:00,685 - INFO - ---


2023-09-05 10:32:00,686 - INFO - Setting logging properties based on config: /workspace/Data/spleen_ct_segmentation/configs/logging.conf.
max epochs: 1


In [7]:
# 2 directly overwriting them after creating the workflow
train_workflow.max_epochs = 3
train_workflow.bundle_root = bundle_root

# Note that must initialize again after changing the content
train_workflow.initialize()
print("max epochs:", train_workflow.max_epochs)
print("bundle root:", train_workflow.bundle_root)

max epochs: 3
bundle root: /workspace/Data


## Using Components in Your Own Pipeline
If you wish to incorporate additional processing into the bundle's existing post-processing and use it within your custom pipeline, you can follow these steps. A comprehensive example can be found [here](https://github.com/Project-MONAI/tutorials/tree/main/model_zoo/app_integrate_bundle).

In [8]:
n_splits = 5
ensemble_transform = MeanEnsembled(keys=["pred"] * n_splits, output_key="pred")
update_postprocessing = Compose((ensemble_transform, train_workflow.val_postprocessing))

print(update_postprocessing.transforms)

(<monai.transforms.post.dictionary.MeanEnsembled object at 0x7f7a40dce160>, <monai.transforms.compose.Compose object at 0x7f7a40e200a0>)


## A simple comparison of the use of `ConfigParser` and `BundleWorkflow`

In [ ]:
# ConfigParser

## load config
bundle_config = ConfigParser()
bundle_config.read_config(config_file)
## update config
bundle_config.config.update({"bundle_root": bundle_root})
## get config
bundle_config.get("bundle_root")
bundle_config.get_parsed_content("train#handlers", instantiate=True)


# BundleWorkflow

## load config
workflow = create_workflow(config_file=config_file, workflow_type="train")
## update config
workflow.bundle_root = bundle_root
workflow.initialize()
## get config
workflow.bundle_root
workflow.train_handlers